In [32]:
import pandas as pd

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:
- 
  - defaults/osx-64::bottleneck==1.2.1=np112py36_0
  - defaults/osx-64::dask==0.14.3=py36_1
  - defaults/osx-64::matplotlib==2.0.2=np112py36_0
  - defaults/osx-64::pywavelets==0.5.2=np112py36_0
  - defaults/osx-64::blaze==0.10.1=py36_0
  - defaults/osx-64::biopython==1.69=np112py36_0
  - defaults/osx-64::distributed==1.16.3=py36_0
  - defaults/osx-64::scikit-image==0.13.0=np112py36_0
  - defaults/osx-64::h5py==2.7.0=np112py36_0
  - defaults/osx-64::statsmodels==0.8.0=np112py36_0
  - defaults/osx-64::pytables==3.3.0=np112py36_0
  - defaults/osx-64::pandas==0.20.1=np112py36_0
  - defaults/osx-64::odo==0.5.0=py36_1
  - defaults/osx-64::numba==0.33.0=np112py36_0
  - defaults/osx-64::seaborn==0.7.1=py36_0
  - defaults/osx-64::astropy==1.3.2=np112py36_0
  - defaults/osx-64::numexpr==2.6.2=np112py36_0
  - defaults/osx-64::anaconda==4.4.0=np1

In [29]:
data = pd.read_csv("speed-dating_raw.csv")
x = ['gender', 'race', 'race_o', 'field']
columns = list(data)

# Deleting Bins
for column in columns:
    if column not in x and data[str(column)].dtype.name == 'object':
        del data[str(column)]

# Deleting useless columns        
del data['has_null']
del data['wave']
del data['d_age']
del data['samerace']
del data['expected_happy_with_sd_people']
del data['expected_num_interested_in_me']
del data['expected_num_matches']
del data['like']
del data['guess_prob_liked']
del data['decision']
del data['decision_o']


# Replace Age NA with mean
mean = round(data['age'].mean())
data['age'].fillna(mean, inplace=True)
mean = round(data['age_o'].mean())
data['age_o'].fillna(mean, inplace=True)

# Make sure difference in age is correct
data['age_d'] = (data['age'] - data['age_o'])
data['age_d_abs'] = data['age_d'].abs()

# Repalce Race NA with Other
data['race'].fillna('other', inplace=True)
data['race_o'].fillna('other', inplace=True)

# Verifying that Same Race is Correct with new Race Names
data['same_race'] = (data['race'] == data['race_o'])

# Replace NA with 0 for Preferences
preferences = ['pref_o_attractive', 'pref_o_sincere', 'pref_o_intelligence', 'pref_o_funny', 'pref_o_ambitious', 'pref_o_shared_interests']          
for pref in preferences:
    data[pref].fillna(0, inplace=True)

# Renaming Column Names
data.rename(columns={'importance_same_race':'same_race_i',
                    'importance_same_religion': 'same_religion_i',
                    'pref_o_attractive':'attractive_o_i',
                    'pref_o_sincere':'sincere_o_i',
                    'pref_o_intelligence':'intelligence_o_i',
                    'pref_o_funny':'funny_o_i',
                    'pref_o_ambitious':'ambitious_o_i',
                    'pref_o_shared_interests':'shared_interests_o_i',
                    'attractive_important':'attractive_i',
                    'sincere_important': 'sincere_i',
                    'intellicence_important': 'intelligence_i',
                    'funny_important':'funny_i',
                    'ambtition_important':'ambitious_i',
                    'shared_interests_important':'shared_interests_i',
                    'ambition':'ambitious',
                    'sinsere_o': 'sincere_o',
                    'ambitous_o':'ambitious_o',
                    'ambition_partner':'ambitious_partner'}, inplace=True)



data['o_i'] = data['attractive_o_i'] + data['sincere_o_i'] + data['intelligence_o_i'] + data['funny_o_i'] + data['ambitious_o_i'] + data['shared_interests_o_i']
data['attractive_o_i'] = (data['attractive_o_i'] / data['o_i'])
data['sincere_o_i'] = (data['sincere_o_i'] / data['o_i'])
data['intelligence_o_i'] = (data['intelligence_o_i'] / data['o_i'])
data['funny_o_i'] = (data['funny_o_i'] / data['o_i'])
data['ambitious_o_i'] = (data['ambitious_o_i'] / data['o_i'])
data['shared_interests_o_i'] = (data['shared_interests_o_i'] / data['o_i'])

data['i'] = data['attractive_i'] + data['sincere_i'] + data['intelligence_i'] + data['funny_i'] + data['ambitious_i'] + data['shared_interests_i']
data['attractive_i'] = (data['attractive_i'] / data['i'])
data['sincere_i'] = (data['sincere_i'] / data['i'])
data['intelligence_i'] = (data['intelligence_i'] / data['i'])
data['funny_i'] = (data['funny_i'] / data['i'])
data['ambitious_i'] = (data['ambitious_i'] / data['i'])
data['shared_interests_i'] = (data['shared_interests_i'] / data['i'])

del data['o_i']
del data['i']

preferences = ['attractive_o_i', 'sincere_o_i', 'intelligence_o_i', 'funny_o_i', 'ambitious_o_i', 'shared_interests_o_i', 'attractive_i', 'sincere_i', 'intelligence_i', 'funny_i', 'ambitious_i', 'shared_interests_i']          
for pref in preferences:
    data[pref].fillna((1.0 / 6.0), inplace = True)



# Replacing same_race_i & same_religion_i with mean
mean = data['same_race_i'].mean()
data['same_race_i'].fillna(round(mean), inplace=True)

mean = data['same_religion_i'].mean()
data['same_religion_i'].fillna(round(mean), inplace=True)


# Replacing Field NA with Mode
mode = data['field'].mode()[0]
data['field'].fillna(mode, inplace=True)

# One Hot Encoding
data = pd.concat([data, pd.get_dummies(data['gender'], prefix = 'gender')], axis = 1)
data = pd.concat([data, pd.get_dummies(data['race'], prefix = 'race')], axis = 1)
data = pd.concat([data, pd.get_dummies(data['race_o'], prefix = 'race_o')], axis = 1)
data = pd.concat([data, pd.get_dummies(data['field'], prefix = 'field')], axis = 1)

del data['gender']
del data['race']
del data['race_o']
del data['field']


# Fill NA's with mean
mean = data['attractive_o'].mean()
data['attractive_o'].fillna(round(mean), inplace = True)
mean = data['sincere_o'].mean()
data['sincere_o'].fillna(round(mean), inplace = True)
mean = data['intelligence_o'].mean()
data['intelligence_o'].fillna(round(mean), inplace = True)
mean = data['funny_o'].mean()
data['funny_o'].fillna(round(mean), inplace = True)
mean = data['ambitious_o'].mean()
data['ambitious_o'].fillna(round(mean), inplace = True)
mean = data['shared_interests_o'].mean()
data['shared_interests_o'].fillna(round(mean), inplace = True)
mean = data['attractive'].mean()
data['attractive'].fillna(round(mean), inplace = True)
mean = data['sincere'].mean()
data['sincere'].fillna(round(mean), inplace = True)
mean = data['intelligence'].mean()
data['intelligence'].fillna(round(mean), inplace = True)
mean = data['funny'].mean()
data['funny'].fillna(round(mean), inplace = True)
mean = data['ambitious'].mean()
data['ambitious'].fillna(round(mean), inplace = True)
mean = data['attractive_partner'].mean()
data['attractive_partner'].fillna(round(mean), inplace = True)
mean = data['sincere_partner'].mean()
data['sincere_partner'].fillna(round(mean), inplace = True)
mean = data['intelligence_partner'].mean()
data['intelligence_partner'].fillna(round(mean), inplace = True)
mean = data['funny_partner'].mean()
data['funny_partner'].fillna(round(mean), inplace = True)
mean = data['ambitious_partner'].mean()
data['ambitious_partner'].fillna(round(mean), inplace = True)
mean = data['shared_interests_partner'].mean()
data['shared_interests_partner'].fillna(round(mean), inplace = True)
mean = data['met'].mean()
data['met'].fillna(round(mean), inplace = True)


# Delete Rows with NA's for Interest Correlate Column
data = data.dropna(axis=0, subset=['interests_correlate'])

data.shape


(8220, 287)

In [33]:
from imblearn.over_sampling import SMOTE
data_copy = data
data_copy['match'] = data_Y
data_copy = df.drop(['match'],axis =1)

sm = SMOTE(sampling_strategy = 'minority')
smote_X, smote_Y = sm.fit_sample(data_copy, data_Y)

print(smote_X.shape)
print(smote_Y.shape)
data = pd.concat([pd.DataFrame(smote_X), pd.Series(smote_Y)], axis=1)
data.head

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
data.to_csv('Speed_Dating_Clean.csv')